In [1]:
#                                           ==============================
#                                           ==============================
#  
#
#                                        Transistor Data Analyzer and Reporter
#                                                      Version: 1.0

#                                           ==============================
#                                           ==============================


# Overview:
# "Transistor Data Analyzer and Reporter" is a  Python-based tool designed for the analysis and reporting 
# of transistor measurement data. This program streamlines the process of collecting data, generating 
# insightful visualizations, and calculating crucial transistor parameters, including mobility and more. Whether you 
# are a physicist, an electronics engineer, a chemist, or simply interested in transistor characteristics, this tool 
# simplifies the analysis process and helps you draw meaningful conclusions from your data.

# Key Features:
# - Data Input: Easily input measurement data for your transistors, whether you're working with individual data points
#   or entire datasets.
# - Data Visualization: Generate informative plots and graphs, including I-V characteristics, transfer characteristics,
#   and more, to visualize the transistor's behavior.
# - Parameter Calculation: Automatically compute essential transistor parameters such as mobility, threshold voltage, 
#   and more.
# - Report Generation: Create professional reports in DOC format that summarize your findings and provide 
#   a clear overview of the transistor's performance.
# - User-Friendly: The program is designed to be user-friendly, making it accessible to both beginners and experts
#   in the field.

# Getting Started:
# 1. Input your transistor measurement data. You can do this by providing a data file.
# 2. Utilize the program's data visualization capabilities to gain insights into your transistor's behavior.
# 3. Enter transistor parameters, such as channel lenght, channel width, dielectric thickness and dielectric coefficient.   
# 4. Let the program automatically calculate key transistor parameters.
# 4. Generate a comprehensive report in DOC format to document your findings.

# Contact Information:
# We encourage collaboration and value your feedback. If you have any questions, suggestions, or you'd like
# to get in touch, please feel free to contact the developer:
# - Developer: Ali Veysel TUNC
# - Email: ali.tunc@kit.edu, aliveyseltunc@gmail.com
# - LinkedIn: https://www.linkedin.com/in/alivtunc/

# Note: This program is open for collaboration and improvement. Your contributions and ideas are welcome, 
# and together we can enhance this tool for the benefit of the Printed Electronics Group (iL).

# Enjoy using the Transistor Data Analyzer and Reporter!

# Copyright © 12.10.2023, Ali TUNC
# ================================================================================================
# ================================================================================================



import tkinter as tk
from tkinter import filedialog, messagebox, ttk
from pandastable import Table, config
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy.stats import linregress
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import screeninfo
import matplotlib.ticker as ticker
from docx import Document 
from docx.shared import Inches
import io
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT, WD_ALIGN_PARAGRAPH
import re
from datetime import datetime


# Constants
epsilon_0 = 8.854187817e-12  # Vacuum permittivity in F/m

default_header_value = "24"  # Metadata for the measurement
default_x_col_value = "2"
default_y_col_value = "6"
default_fitting_start_entry_value = "40"
default_fitting_finish_entry_value = "52"    
default_channel_lenght_entry_value = "80e-6"
default_channel_width_entry_value = "2.5e-3"
default_dielectric_thickness_value = "750e-9"
default_epsilon_r_entry_value = "3.9"
default_vds_entry_value = "5"


# Global variables to store the plot file paths
plot1_file_path = ""
plot2_file_path = ""
plot3_file_path = ""
plot4_file_path = ""
plot5_file_path = "path_to_plot5.png"
plot_output_file_path = "path_to_plot_output"

# Define the regular expression pattern to match both date and time formats
date_time_pattern = r'\d{2}/\d{2}/\d{2} \d{2}:\d{2}:\d{2} [APap][Mm]|\d{2}\.\d{2}\.\d{4} \d{2}:\d{2}:\d{2}'

# All Functions here

def browse_file():
    global date_time_obj
    filetypes1 = (("Text files", "*.txt"), ("CSV files", "*.csv"), ("All files", "*.*"))
    global file_path
    file_path = filedialog.askopenfilename(title="Select a file", filetypes=filetypes1)
    if file_path:
        header_show_data_button.config(state=tk.NORMAL)
        try:
            with open(file_path, 'r') as file:
                text = file.read()
                # Find all matches in the text using the pattern
                matches = re.findall(date_time_pattern, text)
                
                for match in matches:
                    try:
                        date_time_obj = datetime.strptime(match, "%d/%m/%y %I:%M:%S %p")
                    except ValueError:
                        date_time_obj = datetime.strptime(match, "%d.%m.%Y %H:%M:%S")
                    print(date_time_obj)
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {str(e)}")

    
def show_data():
    
    global data, table
    
    header_row = int(header_entry.get())
    data = pd.read_csv(file_path, skiprows=header_row, delimiter="\t")
    #data = data.drop(columns=['Unnamed: 6'])  # drop the Unnamed column
    #data = data.drop([0]).reset_index(drop=True)  # drop the first row to better visualisation
    table_frame = tk.Frame(root)
    table_frame.grid(row=2, column=0, columnspan=4, rowspan=1, padx=20, pady=22, sticky=tk.W)
    table = Table(table_frame, dataframe=data, width=340, height=290, fontsize= 8)
    table.textcolor = 'blue'
    table.cellbackgr = 'white'
    table.boxoutlinecolor = 'black'
    table.fontsize = 2
    
    options = config.load_options()
    options = {'align': 'w',
               'cellbackgr': '#F4F4F3',
               'cellwidth': 55,
               'floatprecision': 2,
               'thousandseparator': '',
               'font': 'Arial',
               'fontsize': 10,
               'fontstyle': '',
               'grid_color': '#ABB1AD',
               'linewidth': 1,
               'rowheight': 22,
               'rowselectedcolor': '#E4DED4', 
               'textcolor': 'black'}
    config.apply_options(options, table)
    
    table.show()
    plot1_button.config(state=tk.NORMAL)

    
def plot1():
    global x, y, plot1_file_path

    x_col = int(x_col_entry.get())
    y_col = int(y_col_entry.get())
    # Sample data
    x = data.iloc[:, x_col-1]
    y = data.iloc[:, y_col-1]
    y1 = data.iloc[:, y_col-1]                              #  y data for normal plot 
    y2 = abs(data.iloc[:, y_col-1])                         #  y data for log plot
    
    # Create the first plot with Y-axis on the left side
    fig, ax1 = plt.subplots(figsize=(4.1,3.4 ))
    fig.set_facecolor("#EFEFEF")
    ax1.plot(x, y1*(-1), 'b-', label='Normal')               # (-1) for better presentation of the plot
    ax1.set_xlabel(r'$\mathrm{V_{gs}[V]}$', fontsize=10)
    ax1.set_ylabel(r'$\mathrm{-I_{ds}[A]}$', color='b',fontsize =10,labelpad=10,rotation=90)
    ax1.tick_params(axis='x', which='both', labelsize=8)
    ax1.tick_params(axis='y', color='b',  labelsize=8,labelcolor='blue')
    # Create the second log plot with Y-axis on the right side
    ax2 = ax1.twinx()
    ax2.plot(x, y2, 'r-', label='Log')
    ax2.set_xlabel(r'$\mathrm{V_{gs}[V]}$', fontsize =10)
    ax2.set_ylabel(r'$\mathrm{I_{ds}[A]}$', color='r',fontsize =10,labelpad=10,rotation=90)
    ax2.tick_params(axis='x', which='both', labelsize=8)
    ax2.tick_params('y', colors='r', labelsize=8, labelcolor='red')
    ax2.set_yscale('log')

    # Display the legends for both plots
    combined_legend = fig.legend(loc='upper right', fontsize=8, labels=['Normal', 'Log'])
    combined_legend.set_bbox_to_anchor((0.78, 0.90))
    plt.tight_layout()
    plt.title('Transfer Characteristic',fontsize =10)
    plt.show()
    plot1_file_path = 'plot1.png'  # Set the file path to save the plot
    fig.savefig(plot1_file_path)
    
    canvas = FigureCanvasTkAgg(fig, master=root)
    canvas.draw()
    canvas.get_tk_widget().grid(row=2, column=4, columnspan=4)   #grid(row=5, column=0, columnspan=4)
 
    fitting_start_entry.config(state=tk.NORMAL)
    fitting_finish_entry.config(state=tk.NORMAL)
    fitting_button.config(state=tk.NORMAL)
    report_button.config(state=tk.NORMAL)
    

def plot2_linearfit():
    global slope, Vgs_0, slope_2, Vgs_0_2, option, y_2, plot2_file_path, plot3_file_path
            
    first_row = int(fitting_start_entry.get())   
    last_row = int(fitting_finish_entry.get())
    
    option = option_var.get()  # Get the selected option
    
    if option == "Linear":
        y_fit = y[first_row:last_row]                         # Calculate slope and intercept from the fitted data
        x_fit = x[first_row:last_row]
        slope, intercept = np.polyfit(x_fit, y_fit, 1)
        Vgs_0 = -intercept / slope

        fig, ax = plt.subplots(figsize=(4.2, 3.4))
        fig.set_facecolor("#EFEFEF")
        ax.plot(x, y*(-1))                                     # to better presentation *(-1)
        ax.plot(x, (slope * x + intercept)*(-1))               # to better presentation *(-1)
        ax.set_ylim(0, np.min(y)*(-1.1))                       # to better presentation *(-1.1)
        ax.set_xlabel(r'$\mathrm{V_{gs}[V]}$', fontsize=10)
        ax.set_ylabel(r'$\mathrm{-I_{ds}[A]}$', fontsize=10)
        ax.tick_params(axis='x', which='both', labelsize=8)
        ax.tick_params(axis='y', color='b', labelsize=8, labelcolor='blue')
        plt.title('Transfer Characteristic in Linear', fontsize=10)
        plt.tight_layout()
        plt.show()
        plot2_file_path = 'plot2.png'  # Set the file path to save the plot
        fig.savefig(plot2_file_path)
        
        fitting_label_result1 = tk.Label(root, text=f"Slope:\t\t\t{slope:.2e}", anchor='w', font=("Arial", 12, "normal"))
        fitting_label_result1.grid(row=3, column=8, padx=10, pady=10, sticky=tk.W, columnspan=2)

        fitting_label_result2 = tk.Label(root, text=f"Threshold Voltage:\t\t{int(Vgs_0)} Volt", anchor='w', font=("Arial", 12, "normal"))
        fitting_label_result2.grid(row=4, column=8, padx=10, pady=10, sticky=tk.W, columnspan=2)

    elif option == "Saturation":
        y_2 =np.sqrt(np.abs(y))     # new dataset for saturation as Ids^0.5
        y_fit_2 = y_2[first_row:last_row]
            # Calculate slope and intercept from the fitted data
        x_fit_2 = x[first_row:last_row]
        slope_2, intercept_2 = np.polyfit(x_fit_2, y_fit_2, 1)
        Vgs_0_2 = (-1)*intercept_2 / slope_2     # slope direction correction (-1)
        
        fig, ax = plt.subplots(figsize=(4.2, 3.3))
        fig.set_facecolor("#EFEFEF")
        ax.plot(x, y_2)                                #  plot the data
        ax.plot(x, slope_2 * x + intercept_2)          #  plot the fitting line
        ax.set_ylim(0, np.max(y_2)*1.1)
        ax.set_xlabel(r'$\mathrm{V_{gs}[V]}$', fontsize=10)
        ax.set_ylabel(r'$\mathrm{I_{ds}^{1/2}[A^{1/2}}]$' , fontsize=10)
        ax.tick_params(axis='x', which='both', labelsize=8)
        ax.tick_params(axis='y', color='b', labelsize=8, labelcolor='blue')
        plt.title('Transfer Characteristic in Saturation', fontsize=10)
        plt.tight_layout()
        plt.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
        plt.show()
        
        plot3_file_path = 'plot3.png'  # Set the file path to save the plot
        fig.savefig(plot3_file_path)
        
        fitting_label_result1 = tk.Label(root, text=f"Slope:\t\t\t{slope_2:.2e}", anchor='w', font=("Arial", 12, "normal"))
        fitting_label_result1.grid(row=3, column=8, padx=10, pady=10, sticky=tk.W, columnspan=2)

        fitting_label_result2 = tk.Label(root, text=f"Threshold Voltage:\t\t{int(Vgs_0_2)} Volt", anchor='w', font=("Arial", 12, "normal"))
        fitting_label_result2.grid(row=4, column=8, padx=10, pady=10, sticky=tk.W, columnspan=2)

    
    canvas = FigureCanvasTkAgg(fig, master=root)
    canvas.draw()
    canvas.get_tk_widget().grid(row=2, column=8, padx=15, pady=10, columnspan=4,sticky=tk.W)
    
    mobility_button.config(state=tk.NORMAL)



def mobility_calculation():
    global epsilon_r, d, L, w, Ci,V_ds, max_mobility_linear_sequantual 
    global plot4_file_path, plot5_file_path, mobility_1, mobility_2
    global new_2_mobility_saturation_sequantual, plot_output_file_path
    option = option_var.get()  # Get the selected option
    
    # Given parameters
    epsilon_r = float(epsilon_r_entry.get())  # Relative permittivity of the gate dielectric
    d = float(dielectric_thickness_entry.get())      # Thickness of the gate dielectric in meters
    L = float(channel_lenght_entry.get())       # Channel length in meters
    w = float(channel_width_entry.get())
    Ci = (epsilon_r * epsilon_0) / d    # Calculate Ci
    
    # Create a frame to group the label
    frame_Ci = tk.Frame(root)
    frame_Ci.grid(row=6, column=3, padx=10, pady=10, sticky=tk.W, columnspan=2)
    # Create a Canvas with a gray background to simulate the frame's color
    canvas_Ci = tk.Canvas(frame_Ci, bg="gray")
    canvas_Ci.pack(fill="both", expand=True)
    # Create the Ci_label and place it inside the frame
    Ci_label = tk.Label(canvas_Ci, text=f"Ci:\t{Ci:.2e} F/m²", bg="white")
    Ci_label.pack()
    
    if option == "Linear":
        V_ds = float(vds_entry.get())       # Drain-source voltage in volts in Linear regime
        mobility_1 = (slope*L)/(w*Ci*V_ds)* 1e4  # *1e4 is for m into cm convertion
        mobility_linear_sequantual = ((y.diff() / x.diff())*L)/(w*Ci*V_ds)* 1e4   # 1e4 is for unit conversion
        max_mobility_linear_sequantual =np.nanmax(mobility_linear_sequantual)
        
        mobility_label_result = tk.Label(root, text=f"Avr. Mobility in LR from Linear Fit\t\t\t{mobility_1:.2e} cm²/(V·s)", anchor='w', font=("Arial", 12, "bold"), width=60, height=1)
        mobility_label_result.grid(row=9, column=6, columnspan=8, padx=10, pady=10, sticky=tk.W, )
        
        
        max_mobility_label_result = tk.Label(root, text=f"Max. Mobility in LR from Sequantual Calculation\t{max_mobility_linear_sequantual:.2e} cm²/(V·s)", anchor='w', font=("Arial", 12, "bold"), width=60, height=1)
        max_mobility_label_result.grid(row=10, column=6, columnspan=8, padx=10, pady=10, sticky=tk.W)
        
        fig, ax = plt.subplots(figsize=(4.2, 3.3))
        fig.set_facecolor("#EFEFEF")
       
        ax.plot(x, mobility_linear_sequantual, marker='o', markersize=3)
        ax.set_xlabel(r'$\mathrm{V_{gs}[V]}$', fontsize=10)
        ax.set_ylabel(r'$\mathrm{µ}[cm^{2}/{V.s}]$' , fontsize=10)
        ax.tick_params(axis='x', which='both', labelsize=8)
        ax.tick_params(axis='y', color='b', labelsize=8, labelcolor='blue')
        plt.title('Mobility in Linear Regime', fontsize=8)
        plt.tight_layout()
        # Set y-axis to scientific notation
        plt.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
        plot4_file_path = 'plot4.png'  # Set the file path to save the plot
        fig.savefig(plot4_file_path)
     
        canvas = FigureCanvasTkAgg(fig, master=root)
        canvas.draw()
        canvas.get_tk_widget().grid(row=2, column=12, columnspan=40, padx=20, pady=10, sticky=tk.W)
        
                
    elif option == "Saturation":
        #V_ds = float(vds_entry.get())                  # Drain-source voltage in volts in Linear regime
        mobility_2 = ((slope_2**2)*2*L)/(w*Ci)* 1e4       # *1e4*1e6 is for m into cm convertion
        mobility_saturation_sequantual = (((y_2.diff() / x.diff())**2)*2*L)/(w*Ci)* 1e4   
        
        new_mobility_saturation_sequantual = mobility_saturation_sequantual[~np.isnan(mobility_saturation_sequantual)]     # get rid of nan values
        new_2_mobility_saturation_sequantual = new_mobility_saturation_sequantual[~np.isinf(new_mobility_saturation_sequantual)]    # get rid of inf values
        
        mobility_label_result = tk.Label(root, text=f"Avr. Mobility in SR from Linear Fit\t\t\t{mobility_2:.2e} cm²/(V·s)", anchor='w', font=("Arial", 12, "bold"), width=60, height=1)
        mobility_label_result.grid(row=9, column=6, padx=10, pady=10, sticky=tk.W, columnspan=8)
        
        max_mobility_label_result = tk.Label(root, text=f"Max. Mobility in SR from from Sequantual Calculation\t{max(new_2_mobility_saturation_sequantual):.2e} cm²/(V·s)", anchor='w', font=("Arial", 12, "bold"), width=60, height=1)
        max_mobility_label_result.grid(row=10, column=6, padx=10, pady=10, sticky=tk.W, columnspan=8)

        fig, ax = plt.subplots(figsize=(4.2, 3.3))
        fig.set_facecolor("#EFEFEF")
             
        ax.plot(x, mobility_saturation_sequantual, marker='*', markersize=3)
        ax.set_xlabel(r'$\mathrm{V_{gs}[V]}$', fontsize=10)
        ax.set_ylabel(r'$\mathrm{µ}[cm^{2}/{V.s}]$' , fontsize=10)
        ax.tick_params(axis='x', which='both', labelsize=8)
        ax.tick_params(axis='y', color='b', labelsize=8, labelcolor='blue')
        plt.title('Mobility in Saturation Regime', fontsize=8)
        plt.tight_layout()
        # Set y-axis to scientific notation
        plt.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
        plot5_file_path = 'plot5.png'  # Set the file path to save the plot
        fig.savefig(plot5_file_path)
        
        canvas = FigureCanvasTkAgg(fig, master=root)
        canvas.draw()
        canvas.get_tk_widget().grid(row=2, column=12, columnspan=40, padx=20, pady=10, sticky=tk.W)
    
    plot_output_button.config(state=tk.NORMAL) 
    

def plot_output():
    global plot_output_file_path
    filetypes2 = (("Text files", "*.txt"), ("CSV files", "*.csv"), ("All files", "*.*"))
    file_path_output = filedialog.askopenfilename(title="Select a file", filetypes=filetypes2)

    df = pd.read_csv(file_path_output, sep='\t', skiprows=24)

    # Create a DataFrame display using Treeview
    table_frame_output = tk.Frame(root)
    table_frame_output.grid(row=1, column=12, rowspan=20, columnspan=40, padx=10, pady=0, sticky=tk.W)
    
    table_output = Table(table_frame_output, dataframe=df, width=360, height=70, fontsize=8)

    # Correct the following lines to set table_output properties:
    table_output.textcolor = 'blue'
    table_output.cellbackgr = 'white'
    table_output.boxoutlinecolor = 'black'
    table_output.fontsize = 8
    table_output.rowheight = 50

    options_output = config.load_options()
    options_output = {
        'align': 'w',
        'cellbackgr': '#F4F4F3',
        'cellwidth': 50,
        'floatprecision': 2,
        'thousandseparator': '',
        'font': 'Arial',
        'fontsize': 10,
        'fontstyle': '',
        'grid_color': '#ABB1AD',
        'linewidth': 1,
        'rowheight': 12,
        'rowselectedcolor': '#E4DED4',
        'textcolor': 'black',
    }
    config.apply_options(options_output, table_output)
    table_output.show()

    canvas_out_plot= tk.Canvas(root, width=430, height=350, bg="#9F9C9C")
    canvas_out_plot.grid(row=6, column=12, columnspan=4, rowspan=6, padx=10, pady=0, sticky=tk.W)
    frame = tk.Frame(canvas_out_plot, bg="#EFEFEF", width=canvas_out_plot.winfo_reqwidth() - 10, height=canvas_out_plot.winfo_reqheight() - 10)
    canvas_out_plot.create_window(5, 5, anchor=tk.NW, window=frame)
    
    data = pd.DataFrame({
        'V_ds': df.iloc[:, 2],
        'I_ds': df.iloc[:, 5],
        r'$\mathrm{V_{gs}[V]}$': df.iloc[:, 1]  # Replace 'Gate [V]' with your actual column name
    })

    # Set Seaborn style and context
    #sns.set_style('whitegrid')
    sns.set_context('notebook')

    # Create a scatter plot with hue
    fig, ax = plt.subplots(figsize=(4.2, 3.3))
    sns.scatterplot(x='V_ds', y='I_ds', hue= r'$\mathrm{V_{gs}[V]}$', data=data, palette='deep', marker='o', s=15, ax=ax)

    # Customize labels and title
    ax.set_xlabel(r'$\mathrm{V_{ds}[V]}$', fontsize=10)
    ax.set_ylabel(r'$\mathrm{I_{ds}[A]}$', fontsize=10)
    ax.tick_params(axis='x', labelsize=8)
    ax.tick_params(axis='y', labelsize=8, labelcolor='blue')
    plt.title('Output Characteristics', fontsize=10)

    # Save the plot
    plot_output_file_path = 'plot_output.png'
    plt.tight_layout()
    plt.savefig(plot_output_file_path)

    # Show the plot
    plt.show()
    fig.patch.set_facecolor('#EFEFEF')

    
    plot_output_file_path = 'plot_output.png'
    fig.savefig(plot_output_file_path)

    canvas = FigureCanvasTkAgg(fig, master=root)
    canvas.draw()
    canvas.get_tk_widget().grid(row=6, column=12, columnspan=40, rowspan=60, padx=15, pady=0, sticky=tk.W)

    root.mainloop()

def create_docx_report():
     # Ask the user for the filename and location to save the DOCX report
    file_path_save_report = filedialog.asksaveasfilename(defaultextension=".docx", filetypes=[("Word Documents", "*.docx")])
    
    if not file_path_save_report:
        # User canceled the file dialog, do nothing
        return
    
    # Extract the file name from the full path
    file_name_save = file_path_save_report.split('/')[-1]  # You might need to adjust this for your system
    
    # Create a new Word document
    doc = Document()
 
    # Create a header and add the file name and 'Table Report' to it
    header = doc.sections[0].header
    table_report_header = header.paragraphs[0]
    table_report_header.alignment = WD_ALIGN_PARAGRAPH.CENTER
    table_report_header.text = file_path_save_report.split('/')[-1]  # File name
    table_report_header_run = table_report_header.runs[0]
    table_report_header_run.font.size = Inches(0.25)

     # Add a title to the main content
    doc.add_heading(f'Report on {file_path}', 0)

    # Select the first 5 and last 5 rows from the DataFrame
    top_rows = data.head(5)
    bottom_rows = data.tail(5)

    # Convert the selected rows to strings
    top_rows_str = top_rows.to_string(index=False)
    bottom_rows_str = bottom_rows.to_string(index=False)

    doc.add_heading(f'Date of the measurement;  {date_time_obj}', 1)

    # Add the table content to the Word document

    heading_data = doc.add_heading('Data', level=1)
    heading_data.alignment = 1  # 1 means centered alignment
    doc.add_paragraph("First 5 Rows:")
    doc.add_paragraph(top_rows_str)
    doc.add_paragraph("Last 5 Rows:")
    doc.add_paragraph(bottom_rows_str)
    doc.add_paragraph(" ")

    heading_dp=doc.add_heading('Device Parameters', 1)
    heading_dp.alignment = 1
    
    doc.add_paragraph(f'ε_r \t:\t{epsilon_r}')
    doc.add_paragraph(f'd \t:\t{d:.2e}(m)')
    doc.add_paragraph(f'Ci \t:\t{Ci:.2e}(F/m²)')
    doc.add_paragraph(f'L \t:\t{L:.2e}(m)')
    doc.add_paragraph(f'W \t:\t{w:.2e}(m)')
    doc.add_paragraph(f'V_ds \t:\t{V_ds} Volt')  
    
    heading_twoaxes= doc.add_heading('Transfer Characteristic in two axes', 1)
    heading_twoaxes.alignment = 1
    doc.add_paragraph("")
    #doc.add_paragraph("Transfer Characteristic in two axes")
    doc.add_picture(plot1_file_path, width=Inches(4))  # Include the saved plot1
 
    heading_output= doc.add_heading('Output Characterstics', 1)
    heading_output.alignment = 1
    doc.add_paragraph("")
    #doc.add_paragraph("Output Characteristic")
    doc.add_picture(plot_output_file_path, width=Inches(4))  
    doc.add_paragraph("")
#     heading_ep=doc.add_heading('Extracted Parameters', 1)
#     heading_ep.alignment = 1
    
    
        
    if option == "Linear":
        heading_ep=doc.add_heading('Extracted Parameters and Plots for Linear Regime', 1)
        heading_ep.alignment = 1             
        #doc.add_heading('Linear Regime', 2)
        doc.add_paragraph(f'Avr. Mobility in Linear Regime from linear fit\t\t\t\t{mobility_1:.2e} cm²/(V·s)')
        doc.add_paragraph(f'Max. Mobility in Linear Regime from sequantual calculation\t\t{max_mobility_linear_sequantual:.2e} cm²/(V·s)')
        doc.add_paragraph(f"Threshold Voltage in Linear Regime from linear fit:\t\t\t{int(Vgs_0)} Volt")
        
#         doc.add_paragraph("")
#         doc.add_paragraph("")
#         doc.add_paragraph("")
#         doc.add_paragraph("")
        
#         heading_plots=doc.add_heading('Plots in Linear Regime', 1)
#         heading_plots.alignment = 1
      
        #doc.add_heading('Linear Regime', 1)
        #doc.add_paragraph("________________________")
        doc.add_paragraph("Transfer Characteristic witted line in Linear Regime")
        doc.add_picture(plot2_file_path, width=Inches(4))  # Include the saved plot2
        #doc.add_heading('', 1)
        doc.add_paragraph("Continious Mobility in Linear Regime")
        doc.add_picture(plot4_file_path, width=Inches(4))  # Include the saved plot4

        
    else:
        heading_ep=doc.add_heading('Extracted Parameters and Plots for Saturation Regime', 1)
        heading_ep.alignment = 1
        #doc.add_heading('Saturation Regime', 2)
        doc.add_paragraph(f'Avr. Mobility in Saturation Regime from linear fit\t\t\t{mobility_2:.2e} cm²/(V·s)')
        doc.add_paragraph(f'Max. Mobility in Saturation Regime from sequantual calculation\t{max(new_2_mobility_saturation_sequantual):.2e} cm²/(V·s)')
        doc.add_paragraph(f"Threshold Voltage in Linear Saturation from linear fit:\t\t{int(Vgs_0_2)} Volt")
        
#         doc.add_paragraph("")
#         doc.add_paragraph("")
#         doc.add_paragraph("")
#         doc.add_paragraph("")
        
#         heading_plots=doc.add_heading('Plots in Saturation Regime', 1)
#         heading_plots.alignment = 1
        

        #doc.add_heading('Saturation Regime', 1)
        #doc.add_paragraph("________________________")
        doc.add_paragraph("Transfer Characteristic witted line in Saturation Regime")
        doc.add_picture(plot3_file_path, width=Inches(4))  # Include the saved plot3
        #doc.add_heading('', 1)
        doc.add_paragraph("Continious Mobility in Saturation Regime")
        doc.add_picture(plot5_file_path, width=Inches(4))  # Include the saved plot5

    # Save the Word document
    doc.save(file_path_save_report)
    
        
        
root = tk.Tk()
root.title("Transistor Data Analyzer and Reporter")
root.lift()

# Get screen width and height
screen = screeninfo.get_monitors()[0]
screen_width = screen.width
screen_height = screen.height

# Set the window size and position
root.geometry(f"{1900}x{1000}+0+0")

# Browse the file 

browse_file_button = tk.Button(root, text="Browse File", command=browse_file)
browse_file_button.grid(row=0, column= 0, padx=10, pady=10, sticky=tk.W)

#First row and headers widgets, Table/Data will be in the second row
header_label = tk.Label(root, text="Number of Header Lines ")
header_label.grid(row=1, column=0, padx=10, pady=10, sticky=tk.W)
header_entry = tk.Entry(root, state=tk.NORMAL, width=5)
header_entry.grid(row=1, column=1, padx=10, pady=10, sticky=tk.W)
header_entry.insert(0, default_header_value)

header_show_data_button = tk.Button(root, text="Show Data", command=show_data, state=tk.DISABLED)
header_show_data_button.grid(row=1, column=2, padx=10, pady=10, sticky=tk.W)

# 2th row table and plot canvas
canvas_table_1= tk.Canvas(root, width=420, height=350, bg="#9F9C9C")
canvas_table_1.grid(row=2, column=0, columnspan=4, rowspan=1, padx=10, pady=10, sticky=tk.W)
frame = tk.Frame(canvas_table_1, bg="#EFEFEF", width=canvas_table_1.winfo_reqwidth() - 10, height=canvas_table_1.winfo_reqheight() - 10)
canvas_table_1.create_window(5, 5, anchor=tk.NW, window=frame)


canvas_plot_1= tk.Canvas(root, width=420, height=350, bg="#9F9C9C")
canvas_plot_1.grid(row=2, column=4, columnspan=4, padx=10, pady=10, sticky=tk.W)
frame = tk.Frame(canvas_plot_1, bg="#EFEFEF", width=canvas_plot_1.winfo_reqwidth() - 10, height=canvas_plot_1.winfo_reqheight() - 10)
canvas_plot_1.create_window(5, 5, anchor=tk.NW, window=frame)


# 3th row, select x column 

x_col_label = tk.Label(root, text="X Column")
x_col_label.grid(row=3,column=0, padx=10, pady=10, sticky=tk.W)
x_col_entry = tk.Entry(root, width=5)
x_col_entry.grid(row=3,column=1, padx=10, pady=10, sticky=tk.W)
x_col_entry.insert(0, default_x_col_value)

#4th row select y column and plot button for ploting first simple plot

y_col_label = tk.Label(root, text="Y Column")
y_col_label.grid(row=4,column=0, padx=10, pady=10, sticky=tk.W)
y_col_entry = tk.Entry(root, width=5)
y_col_entry.grid(row=4,column=1, padx=10, pady=10, sticky=tk.W)
y_col_entry.insert(0, default_y_col_value)

plot1_button= tk.Button(root, text="Plot simple Vgs-Ids", command =plot1, state=tk.DISABLED)
plot1_button.grid(row=4, column=2, padx=0, pady=10, sticky=tk.E)

#5th row
# default_fitting_start_entry_value = "50"
# default_fitting_finish_entry_value = "60" 
fitting_label_start = tk.Label(root, text="Fit Data start Point  ")
fitting_label_start.grid(row=3, column=4, padx=10, pady=10,sticky=tk.W)  

fitting_start_entry = tk.Entry(root, state=tk.NORMAL, width=5)
fitting_start_entry.grid(row=3, column=5, padx=10, pady=10)
fitting_start_entry.insert(0, default_fitting_start_entry_value )

fitting_label_finish = tk.Label(root, text="Fit Data Finish Point  ")
fitting_label_finish.grid(row=4, column=4, padx=10, pady=10, sticky=tk.W)   #(row=7, column=0, padx=10, pady=10)

fitting_finish_entry = tk.Entry(root, state=tk.NORMAL, width=5)
fitting_finish_entry.grid(row=4, column=5, padx=10, pady=10)
fitting_finish_entry.insert(0, default_fitting_finish_entry_value)

fitting_button = tk.Button(root, text="Perform Linear Fit", width=15, command=plot2_linearfit, state=tk.DISABLED)
fitting_button.grid(row=4, column=6, columnspan=2, padx=0, pady=10, sticky=tk.W)


# 2th plot and plot canvas
canvas_plot_2= tk.Canvas(root, width=430, height=350, bg="#9F9C9C")
canvas_plot_2.grid(row=2, column=8, columnspan=4, padx=10, pady=10, sticky=tk.W)
frame = tk.Frame(canvas_plot_2, bg="#EFEFEF", width=canvas_plot_2.winfo_reqwidth() - 10, height=canvas_plot_2.winfo_reqheight() - 10)
canvas_plot_2.create_window(5, 5, anchor=tk.NW, window=frame)

# 3th plot and plot canvas
canvas_plot_3= tk.Canvas(root, width=430, height=350, bg="#9F9C9C")
canvas_plot_3.grid(row=2, column=12, columnspan=40, padx=10, pady=10, sticky=tk.W)
frame = tk.Frame(canvas_plot_3, bg="#EFEFEF", width=canvas_plot_2.winfo_reqwidth() - 10, height=canvas_plot_2.winfo_reqheight() - 10)
canvas_plot_3.create_window(5, 5, anchor=tk.NW, window=frame)


mobility_button = tk.Button(root, text="Calculate Mobility", command=mobility_calculation, state=tk.DISABLED)
mobility_button.grid(row=8, column=6, columnspan=3, padx=0, pady=1, sticky=tk.W)

plot_output_label = tk.Label(root, text = "Would you like to add an output characteristic to the report?")
plot_output_label.grid(row=7, column=8, padx=0, pady=10, columnspan=3, sticky=tk.E)
plot_output_button= tk.Button(root, text="Add corresponding Output", command =plot_output, state=tk.DISABLED)
plot_output_button.grid(row=8, column=8, padx=0, pady=10, sticky=tk.E)

report_button = tk.Button(root, text="Create Peroformance Report", command=create_docx_report, state=tk.DISABLED)
report_button.grid(row=12, column=6, columnspan=3, padx=0, pady=10, sticky=tk.W)

Empty_label = tk.Label(root, text="     ")
Empty_label.grid(row=5, column=1, padx=10, pady=10)

epsilon_r_label = tk.Label(root, text="ε_r ")
epsilon_r_label.grid(row=6, column=0, padx=10, pady=10,sticky=tk.W)
epsilon_r_entry = tk.Entry(root, state=tk.NORMAL, width=10)
epsilon_r_entry.grid(row=6, column=1, padx=10, pady=10,sticky=tk.W)
epsilon_r_entry.insert(0, default_epsilon_r_entry_value )

dielectric_thickness_label = tk.Label(root, text="Dielectric Thickness (m)")
dielectric_thickness_label.grid(row=7, column=0, padx=10, pady=10,sticky=tk.W)
dielectric_thickness_entry = tk.Entry(root, state=tk.NORMAL, width=10)
dielectric_thickness_entry.grid(row=7, column=1, padx=10, pady=10,sticky=tk.W)
dielectric_thickness_entry.insert(0, default_dielectric_thickness_value )

channel_lenght_label = tk.Label(root, text="Channel Lenght(m)")
channel_lenght_label.grid(row=8, column=0, padx=10, pady=10,sticky=tk.W)
channel_lenght_entry = tk.Entry(root, state=tk.NORMAL, width=10)
channel_lenght_entry.grid(row=8, column=1, padx=10, pady=10,sticky=tk.W)
channel_lenght_entry.insert(0, default_channel_lenght_entry_value )

channel_width_label = tk.Label(root, text="Channel Width(m)")
channel_width_label.grid(row=9, column=0, padx=10, pady=10,sticky=tk.W)
channel_width_entry = tk.Entry(root, state=tk.NORMAL, width=10)
channel_width_entry.grid(row=9, column=1, padx=10, pady=10,sticky=tk.W)
channel_width_entry.insert(0, default_channel_width_entry_value )

vds_label = tk.Label(root, text="Vds")
vds_label.grid(row=10, column=0, padx=10, pady=10,sticky=tk.W)
vds_entry = tk.Entry(root, state=tk.NORMAL, width=10)
vds_entry.grid(row=10, column=1, padx=10, pady=10,sticky=tk.W)
vds_entry.insert(0, default_vds_entry_value )



#option_label = tk.Label(root, text="Select the regime")
#option_label.pack()
#option_label.grid(row=5, column=4, padx=10, pady=10,sticky=tk.E)

option_var = tk.StringVar(value="Linear")
option_menu = tk.OptionMenu(root, option_var, "Linear", "Saturation")
option_menu.config(width=11)            # Set the width of the button

#option_menu.pack()
option_menu.grid(row=3, column=6, padx=0, pady=10,sticky=tk.W)




root.mainloop()

In [ ]:
# Transistor Data Analyzer and Reporter
# Version: 1.0

# Overview:
# "Transistor Data Analyzer and Reporter" is a  Python-based tool designed for the analysis and reporting 
# of transistor measurement data. This program streamlines the process of collecting data, generating 
# insightful visualizations, and calculating crucial transistor parameters, including mobility and more. Whether you 
# are a physicist, an electronics engineer, a chemist, or simply interested in transistor characteristics, this tool 
# simplifies the analysis process and helps you draw meaningful conclusions from your data.

# Key Features:
# - Data Input: Easily input measurement data for your transistors, whether you're working with individual data points
#   or entire datasets.
# - Data Visualization: Generate informative plots and graphs, including I-V characteristics, transfer characteristics,
#   and more, to visualize the transistor's behavior.
# - Parameter Calculation: Automatically compute essential transistor parameters such as mobility, threshold voltage, 
#   and more.
# - Report Generation: Create professional reports in DOC format that summarize your findings and provide 
#   a clear overview of the transistor's performance.
# - User-Friendly: The program is designed to be user-friendly, making it accessible to both beginners and experts
#   in the field.

# Getting Started:
# 1. Input your transistor measurement data. You can do this by providing a data file.
# 2. Utilize the program's data visualization capabilities to gain insights into your transistor's behavior.
# 3. Enter transistor parameters, such as channel lenght, channel width, dielectric thickness and dielectric coefficient.   
# 4. Let the program automatically calculate key transistor parameters.
# 4. Generate a comprehensive report in DOC format to document your findings.

# Contact Information:
# We encourage collaboration and value your feedback. If you have any questions, suggestions, or you'd like
# to get in touch, please feel free to contact the developer:
# - Developer: Ali Veysel TUNC
# - Email: ali.tunc@kit.edu, aliveyseltunc@gmail.com
# - LinkedIn: https://www.linkedin.com/in/alivtunc/

# Note: This program is open for collaboration and improvement. Your contributions and ideas are welcome, 
# and together we can enhance this tool for the benefit of the Printed Electronics Group (iL).

# Enjoy using the Transistor Data Analyzer and Reporter!

# Copyright © 12.10.2023, Ali TUNC


In [ ]:
plot_file_path

In [ ]:
table

In [ ]:
import matplotlib.pyplot as plt

# Başlangıç performansı (örneğin, %100 olarak kabul edelim)
initial_performance = 1

# Yılın gün sayısı
days_in_year = 365

# Günlük performans artışı yüzdesi
daily_increase_percentage = 1

# Performansı saklamak için boş bir liste oluşturun
performance_data = []

# Performans hesaplama ve veri toplama
for day in range(days_in_year + 1):
    performance_data.append(initial_performance)
    initial_performance *= (1 + daily_increase_percentage / 100)

# Veriyi grafiğe çizdirme
plt.plot(range(days_in_year + 1), performance_data)
plt.title('Yıllık Performans Değişikliği')
plt.xlabel('Gün')
plt.ylabel('Performans')
plt.grid(True)
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Başlangıç performansı (örneğin, %100 olarak kabul edelim)
initial_performance = 1

# Yılın gün sayısı
days_in_year = 365

# Günlük performans artışı yüzdesi
daily_decrease_percentage = 1

# Performansı saklamak için boş bir liste oluşturun
performance_data = []

# Performans hesaplama ve veri toplama
for day in range(days_in_year + 1):
    performance_data.append(initial_performance)
    initial_performance *= (1 - daily_increase_percentage / 100)

# Veriyi grafiğe çizdirme
plt.plot(range(days_in_year + 1), performance_data)
plt.title('Yıllık Performans Değişikliği')
plt.xlabel('Gün')
plt.ylabel('Performans')
plt.grid(True)
plt.show()


In [ ]:
while True:
    try:
        # Kullanıcıdan bir giriş iste
        girdi = input("Lütfen bir sayı girin: ")
        # Girişi tam sayıya dönüştürmeye çalış
        sayi = int(girdi)
        # Eğer dönüştürme işlemi hatasız olduysa döngüden çık
        break
    except ValueError:
        # Eğer dönüştürme işlemi başarısız olursa kullanıcıya uyarı ver ve tekrar giriş iste
        print("Lütfen bir sayı girin!")

print("Girdiğiniz sayı:", sayi)
